In [82]:
from flask import Flask, request
from nornir import InitNornir
import pynetbox
#from nornir.core.inventory import Host
from ipaddress import IPv4Address
from ipaddress import IPv4Network
import ipaddress
from jinja2 import Environment, FileSystemLoader

In [60]:
nr = InitNornir(
    inventory={
        "plugin": "NetBoxInventory2",
        "options": {
            "nb_url": "http://10.30.1.101:8000",
            "nb_token": "dc5b766afda60e647a9fcc384c736a9689da8618",
            "group_file": "./inventory/groups.yml",
            "defaults_file": "./inventory/defaults.yml",
        },
    },
)

In [61]:
nb_url = nr.config.inventory.options['nb_url']
nb_token = nr.config.inventory.options['nb_token']
nb = pynetbox.api(
    nb_url,
    token=nb_token
)

In [62]:
nb_site = 'plgn'
get_all_devices = list(nb.dcim.devices.filter(site = nb_site))
#get_all_devices

In [63]:
# List all IPs prefixes from all sites
nb_site = 'plgn'
nb_role = 'plgn'
IPv4 = 4
get_all_ip_prefixes = list(nb.ipam.prefixes.filter(site = nb_site, role = nb_role))
for ip_prefix in get_all_ip_prefixes:
    if ip_prefix.family.value == IPv4:
        print(ip_prefix.description)
        print(ip_prefix)

Servers pool
10.30.1.0/24
Servers test pool
10.30.3.0/24


In [64]:
# List all IPs from all devices
get_all_ip_addresses = list(nb.ipam.ip_addresses.filter(device=get_all_devices))
#get_all_ip_addresses

In [65]:
# List all interfaces from all devices
get_all_interfaces = list(nb.dcim.interfaces.filter(device=get_all_devices))
#get_all_interfaces

In [66]:

templates_path = "./templates/"
environment = Environment(loader=FileSystemLoader(templates_path))
template = environment.get_template("dhcpd_static.template")
for device_ip in get_all_ip_addresses:
        for ip_prefix in get_all_ip_prefixes:
                if IPv4Address(device_ip.address.split('/')[0]) in IPv4Network(ip_prefix.prefix):
                        for device_interface in get_all_interfaces:
                                if device_ip.assigned_object.device.name == device_interface.device.name and device_interface.mac_address != None:
                                        content = template.render(
                                        device_name = device_interface.device.name,
                                        mac_address = device_interface.mac_address,
                                        ip_address = device_ip.address.split('/')[0]
                                        )
                                        print(content)

host SRV-PEV-5 {
  hardware ethernet 2A:2F:0A:9F:A8:B5;
  fixed-address 10.30.1.248;
}


In [113]:
#Create a Flask instance
""" app = Flask(__name__)

@app.route('/api/ip_address', methods=['POST'])
def webhook():
    device_ip =  request.json["data"]["address"]
    device = request.json["data"]["assigned_object"]["device"]["name"]
    interface = request.json["data"]["assigned_object"]["name"]
    #print(ip.ip,ip.netmask,ip.network,ip.with_prefixlen,device)
    #print(request.json)
    
    templates_path = "./templates/"
    environment = Environment(loader=FileSystemLoader(templates_path))
    template = environment.get_template("dhcpd_static.template")

    for ip_prefix in get_all_ip_prefixes:
            if ipaddress.ip_interface(device_ip) in IPv4Network(ip_prefix.prefix):
                for device_interface in interface:
                    if device_ip.assigned_object.device.name == device_interface.device.name and device_interface.mac_address != None:
                        content = template.render(
                        device_name = device_interface.device.name,
                        mac_address = device_interface.mac_address,
                        ip_address = device_ip.address.split('/')[0]
                         )
                        print(content)
    
    return "Webhook received!"

app.run(host='0.0.0.0', port=8080)
if __name__ == "__main__": app.run() """

ip = ['10.30.1.101/24']
prefix = ['10.30.1.0/24']
interface = 'iDRAC'
device = 'SRV-PEV-1'
templates_path = "./templates/"
environment = Environment(loader=FileSystemLoader(templates_path))
template = environment.get_template("dhcpd_static.template")
device_interface = list(nb.dcim.interfaces.filter(device=device, name=interface))
for device_ip in ip:
   # print(ipaddress.ip_interface(device_ip).ip)
    content = template.render(
    device_name = device,
    mac_address = device_interface[0].mac_address,
    ip_address = ipaddress.ip_interface(device_ip).ip
    )
    print(content)

host SRV-PEV-1 {
  hardware ethernet None;
  fixed-address 10.30.1.101;
}
